In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('../input/housepricing/HousePrices_HalfMil.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

## Data Visualization

In [ ]:
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns

In [ ]:
plt.figure(figsize=(20,20)) 
sns.heatmap(df.corr(), annot=True, cmap = 'RdBu', center=0);

In [ ]:
# price distribution
plt.hist(df['Prices'],bins=30, color ='green',density= True,edgecolor = 'red',label="hist")
kde = stats.kde.gaussian_kde(df['Prices'])
x = np.linspace(0, 100000, 100)
y = kde(x)
plt.plot(x,y,c="black",linewidth=3,label="k-density")
plt.xlabel('Sale price')
plt.legend()
plt.show()

In [ ]:
l=list()
for i in range(1,4):
    l.append(df[df['City']==i]['Prices'])
plt.boxplot(l)
plt.xlabel('City')
plt.ylabel('Price')
plt.ylim(0,100000)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,6))
sns.lineplot(x = df["Area"] , y = df["Prices"])


In [ ]:
fig = plt.figure(figsize=(15,6))


color=['blue','red','green']
for i in range(1,4):
    plt.scatter(df[df['City']==i]['Area'],df[df['City']==i]['Prices'],color=color[i-1],label=i, alpha = 0.5)
    plt.legend()
    plt.xlabel("Area")
    plt.ylabel("Price")

plt.show()

In [ ]:
fig = plt.figure(figsize=(15,6))


color=['blue','red','green']
for i in range(1,4):
    plt.scatter(df[df['City']==i]['Floors'],df[df['City']==i]['Prices'],color=color[i-1],label=i, alpha = 0.5)
    plt.legend()
    plt.xlabel("Floors")
    plt.ylabel("Price")

plt.show()

## Multiple Linear Regression

In [ ]:
from sklearn import model_selection, preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,cross_val_score

target = df['Prices']
data = df.drop('Prices', axis =1)

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=789)

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)


In [ ]:
#Display the intercept as well as the coefficients of each variable estimated by the model

coeffs = list(lr.coef_)
coeffs.insert(0, lr.intercept_)

feats = list(data.columns)
feats.insert(0, 'intercept')

pd.DataFrame({'valeur estimée': coeffs}, index = feats)

In [ ]:
print("Coefficient de détermination du modèle :", lr.score(X_train, y_train))
print("Coefficient de détermination obtenu par Cv :", cross_val_score(lr,X_train,y_train).mean())

In [ ]:
lr.score(X_test,y_test)

In [ ]:
#Display in a graph the scatter plot between pred_test and y_test
pred_test = lr.predict(X_test)
plt.scatter(pred_test, y_test)
plt.plot((y_test.min(),y_test.max()), (y_test.min(),y_test.max()))

### The points are perfectly close to this line, so the prediction is very good.

In [ ]:
#Display the scatter plot representing the residuals as a function of the values of y_train.
pred_train = lr.predict(X_train)
residus = pred_train - y_train

plt.scatter(y_train, residus, color = '#980a10', s=15)
plt.plot((y_train.min(),y_train.max()), (0,0), lw=3, color = '#0a5798')

### The probplot function of scipy.stats makes it possible to display a Quantile-Quantile plot or (Q-Q plot) which makes it possible to evaluate the relevance of the fit of a given distribution to a theoretical model (often a reduced centered Gaussian law). Thus the normality of the residuals (once centered reduced) is easily validated: if the points are aligned on the first bisector, it is because the distribution of the residuals probably follows a normalized Gaussian law.

In [ ]:
residus_norm = (residus-residus.mean())/residus.std()

stats.probplot(residus_norm, plot=plt)

plt.show()

### The normality assumption is plausible, with the points lining up approximately along the line.

### Thanks for reading!